xclim
https://xclim.readthedocs.io/en/stable/indicators.html

CEDA dataset
https://catalogue.ceda.ac.uk/uuid/6a44fecc0f3842faaea53ab617dd2047/?q=&sort_by=title_asc&results_per_page=20&objects_related_to_uuid=6a44fecc0f3842faaea53ab617dd2047&&record_type=Observation&jump=related-anchor

CEDA Download
https://dap.ceda.ac.uk/badc/ukcp18/data/land-cpm/uk/5km/rcp85/01/

UKCP18 Doc
https://www.metoffice.gov.uk/binaries/content/assets/metofficegovuk/pdf/research/ukcp/ukcp18-guidance-data-availability-access-and-formats.pdf

In [5]:
import xarray as xr
import rioxarray
import os
import glob
import pandas as pd
import geopandas as gpd
import numpy as np
from rasterio import features
import rasterio
import os
from shapely.geometry import shape, box, mapping
from shapely.geometry.polygon import orient
import json
import xclim.indicators.atmos as xci
import sys
from collections import defaultdict
import warnings
import time
warnings.filterwarnings("ignore", category=UserWarning, module="xclim")

In [2]:
stats = ["max", "min", "med"]
variants = ["tas", "tasmas", "tasmin"]
rolling_period = 30
output_folder = "./processed/tas"
input_folder = "./raw/tas"
models = ["01", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "15", "23", "25", "27", "29"]

### Set up metrics

In [6]:
def fix_metric(metric, merged_ds):
    metric = metric.groupby('time.year').mean(dim='time', keep_attrs=True)
    metric = metric.isel(year=slice(1, None))
    metric = metric.transpose(*merged_ds['min_temp'].dims)
    metric = metric.assign_coords({
            "projection_y_coordinate": merged_ds.projection_y_coordinate,
            "projection_x_coordinate": merged_ds.projection_x_coordinate,
            "year": merged_ds.year
        })
    metric.attrs.update(merged_ds["min_temp"].attrs)
    return metric

def combine_and_extract(model_number):
    start_time = time.time()
    print("Started loading files:", time.time() - start_time, "seconds")
    print(f"Processing model {model_number}")
    tas_files = glob.glob(os.path.join(input_folder,model_number, "tas_*.nc"))
    tasmax_files = glob.glob(os.path.join(input_folder,model_number, "tasmax_*.nc"))
    tasmin_files = glob.glob(os.path.join(input_folder,model_number, "tasmin_*.nc"))
    all_tas = tas_files + tasmax_files + tasmin_files
    combined_ds = xr.open_mfdataset(all_tas, combine='by_coords')

    yearly_groups = []
    for i, (year, yearly_ds) in enumerate(combined_ds.groupby('time.year')):
        augmented_yearly_data = yearly_ds.mean(dim='time', keep_attrs=True)        
        augmented_yearly_data["min_temp"] = yearly_ds['tasmin'].min(dim='time', keep_attrs=True)
        augmented_yearly_data["max_temp"] = yearly_ds['tasmax'].max(dim='time', keep_attrs=True)
        yearly_groups.append(augmented_yearly_data)
    
    print("Processed yearly groups:", time.time() - start_time, "seconds")
    yearly_groups = yearly_groups[1:]
    merged_ds = xr.concat(yearly_groups, dim='year')
    combined_ds.close()

    merged_ds['rolling_min_temp'] = merged_ds['min_temp'].rolling(year=30, center=True, min_periods=1).mean()
    merged_ds['rolling_max_temp'] = merged_ds['max_temp'].rolling(year=30, center=True, min_periods=1).mean()

    cdda=xci.cooling_degree_days_approximation(
        tas=combined_ds['tas'], 
        tasmax=combined_ds['tasmax'],
        tasmin=combined_ds['tasmin'],
        freq='YS',
        thresh="20.0 degC"
    )
    merged_ds['cooling_degree_days'] = fix_metric(cdda, merged_ds)
    merged_ds['rolling_cdd'] = merged_ds['cooling_degree_days'].rolling(year=rolling_period, center=True, min_periods=1).mean()
    # print("Processed cooling degree days:", time.time() - start_time, "seconds")
    
    hdda=xci.heating_degree_days_approximation(
        tas=combined_ds['tas'], 
        tasmax=combined_ds['tasmax'],
        tasmin=combined_ds['tasmin'],
        freq='YS',
        thresh="15.5 degC"
    )
    merged_ds['heating_degree_days'] = fix_metric(hdda, merged_ds)
    merged_ds['rolling_hdd'] = merged_ds['heating_degree_days'].rolling(year=rolling_period, center=True, min_periods=1).mean()
    # print("Processed heating degree days:", time.time() - start_time, "seconds")

    hwf = xci.heat_wave_frequency(
        tasmax=combined_ds['tasmax'],
        tasmin=combined_ds['tasmin'],
        freq='YS',
        thresh_tasmax="26.5 degC",
        thresh_tasmin='10.0 degC',
        window=3
    )
    merged_ds['heatwave_freq'] = fix_metric(hwf, merged_ds)
    merged_ds['rolling_hw_freq'] = merged_ds['heatwave_freq'].rolling(year=rolling_period, center=True, min_periods=1).mean()
    # print("Processed heatwave frequency:", time.time() - start_time, "seconds")

    hwl = xci.heat_wave_max_length(
        tasmax=combined_ds['tasmax'],
        tasmin=combined_ds['tasmin'],
        freq='YS',
        thresh_tasmax="26.5 degC",
        thresh_tasmin='10.0 degC',
    )
    merged_ds['heatwave_length'] = fix_metric(hwl, merged_ds)
    merged_ds['rolling_hw_length'] = merged_ds['heatwave_length'].rolling(year=rolling_period, center=True, min_periods=1).mean()
    # print("Processed heatwave length:", time.time() - start_time, "seconds")

    cfd = xci.consecutive_frost_days(
        tasmin=combined_ds['tasmin'],
    )

    merged_ds['conseq_frost_days'] = fix_metric(cfd, merged_ds)
    merged_ds['rolling_cfd'] = merged_ds['conseq_frost_days'].rolling(year=rolling_period, center=True, min_periods=1).mean()
    print("Completed metrics:", time.time() - start_time, "seconds")

    merged_ds = merged_ds.drop_vars(['tas', 'tasmax', 'tasmin', 'conseq_frost_days', 'heatwave_length', 'heatwave_freq','heating_degree_days', 'cooling_degree_days','min_temp','max_temp'])
    out_rolling_path = os.path.join(output_folder, '01_rolling', f"rolling_average_{model_number}.nc")
    merged_ds.to_netcdf(out_rolling_path)

In [14]:
select_models = models[14:16]

for model in select_models:
    combine_and_extract(model)

Started loading files: 0.0 seconds
Processing model 27
Processed yearly groups: 7.474636554718018 seconds
Completed metrics: 26.47073006629944 seconds
Started loading files: 0.0 seconds
Processing model 29
Processed yearly groups: 6.881790399551392 seconds
Completed metrics: 28.939653873443604 seconds


### Characterise data

In [15]:
def characterise_rolling_averages():
    # Step 1: Load all NetCDF files
    file_paths = glob.glob("./processed/tas/01_rolling/*.nc")  # Replace with your actual directory

    # Step 2: Open them as a multi-file dataset (combine along 'ensemble_member')
    datasets = [xr.open_dataset(fp, decode_times=False) for fp in file_paths]
    combined = xr.concat(datasets, dim='ensemble_member')

    # Step 3: Compute statistics across the ensemble dimension
    median_ds = combined.median(dim='ensemble_member', keep_attrs=True)
    min_ds = combined.min(dim='ensemble_member', keep_attrs=True)
    max_ds = combined.max(dim='ensemble_member', keep_attrs=True)

    # Step 4: Save each result to new .nc files
    median_ds.to_netcdf("./processed/tas/01-1_stats/temp_median.nc")
    min_ds.to_netcdf("./processed/tas/01-1_stats/temp_min.nc")
    max_ds.to_netcdf("./processed/tas/01-1_stats/temp_max.nc")

In [16]:
characterise_rolling_averages()

### Manual processing

Before running the next steps, load the created .nc file into QGIS, use the batch function to clip the layers, without manually reprojecting them

In [17]:
def raster_cells_to_features(image, transform):
    features = []
    rows, cols = image.shape
    for row in range(rows):
        for col in range(cols):
            value = image[row, col]
            if np.isnan(value):
                continue
            # Get the bounds of the cell
            x0, y0 = rasterio.transform.xy(transform, row, col, offset='ul')
            x1, y1 = rasterio.transform.xy(transform, row, col, offset='lr')
            geom = box(x0, y1, x1, y0)
            features.append({
                'geometry': mapping(geom),
                'properties': {'value': float(value)}
            })
    return features

def tif_to_geojson(tif_folder, geojson_folder):
    absYears = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
    os.makedirs(geojson_folder, exist_ok=True)
    tif_files = glob.glob(os.path.join(tif_folder, "*.tif"))
    
    for tif_file in tif_files:
        years = absYears
        with rasterio.open(tif_file) as src:
            print(f"\nProcessing {tif_file}")

            if src.count > 1:
                timebands = [src.descriptions[i] if src.descriptions[i] else f"{i+1}" for i in range(src.count)]
            else:
                timeband = src.tags().get('timeband', None)
                timebands = [timeband if timeband else "unknown"]

            selected_indices = [i for i, tb in enumerate(timebands) if any(str(y) in str(tb) for y in years)]

            for band_idx in selected_indices:
                image = src.read(band_idx + 1)

                if src.nodata is not None:
                    mask = ~np.isnan(image) & (image != src.nodata)
                else:
                    mask = ~np.isnan(image)
                geoms = raster_cells_to_features(image, src.transform)
                gdf = gpd.GeoDataFrame.from_features(geoms, crs=src.crs)
                gdf = gdf.to_crs("EPSG:4326")
                geojson_path = os.path.join(
                    geojson_folder,
                    f"{os.path.splitext(os.path.basename(tif_file))[0]}_{timebands[band_idx]}.geojson"
                )
                gdf.to_file(geojson_path, driver="GeoJSON")


In [18]:
# Converts the reprojected and clipped tif files into geojson files at the different time steps

tif_to_geojson(os.path.join(output_folder, "02_clipped"), os.path.join(output_folder, "03_geojson"))


Processing ./processed/tas\02_clipped\TAS_max_cdd.tif

Processing ./processed/tas\02_clipped\TAS_max_cfd.tif

Processing ./processed/tas\02_clipped\TAS_max_hdd.tif

Processing ./processed/tas\02_clipped\TAS_max_hwfreq.tif

Processing ./processed/tas\02_clipped\TAS_max_hwlen.tif

Processing ./processed/tas\02_clipped\TAS_max_maxT.tif

Processing ./processed/tas\02_clipped\TAS_max_minT.tif

Processing ./processed/tas\02_clipped\TAS_med_cdd.tif

Processing ./processed/tas\02_clipped\TAS_med_cfd.tif

Processing ./processed/tas\02_clipped\TAS_med_hdd.tif

Processing ./processed/tas\02_clipped\TAS_med_hwfreq.tif

Processing ./processed/tas\02_clipped\TAS_med_hwlen.tif

Processing ./processed/tas\02_clipped\TAS_med_maxT.tif

Processing ./processed/tas\02_clipped\TAS_med_minT.tif

Processing ./processed/tas\02_clipped\TAS_min_cdd.tif

Processing ./processed/tas\02_clipped\TAS_min_cfd.tif

Processing ./processed/tas\02_clipped\TAS_min_hdd.tif

Processing ./processed/tas\02_clipped\TAS_min_hwfr

### Inverts JSONS
Inverts the storage of data so that it uses lat/lon as keys to reference all the values temporaly

In [19]:
def inverseGeojson(filename, stat):
    file_paths = [
    os.path.join(output_folder, "03_geojson", f"{filename}_10.geojson"),
    os.path.join(output_folder, "03_geojson", f"{filename}_20.geojson"),
    os.path.join(output_folder, "03_geojson", f"{filename}_30.geojson"),
    os.path.join(output_folder, "03_geojson", f"{filename}_40.geojson"),
    os.path.join(output_folder, "03_geojson", f"{filename}_50.geojson"),
    os.path.join(output_folder, "03_geojson", f"{filename}_60.geojson"),
    os.path.join(output_folder, "03_geojson", f"{filename}_70.geojson"),
    os.path.join(output_folder, "03_geojson", f"{filename}_80.geojson"),
    os.path.join(output_folder, "03_geojson", f"{filename}_90.geojson"),
    os.path.join(output_folder, "03_geojson", f"{filename}_100.geojson"),
]

    file_labels = [
        "1990",
        "2000",
        "2010",
        "2020",
        "2030",
        "2040",
        "2050",
        "2060",
        "2070",
        "2080"
    ]

    # Dictionary to store: { coordinate: [value1, value2, ...] }
    coordinate_data = defaultdict(lambda: [None] * len(file_paths))

    # Process each file
    for idx, file_path in enumerate(file_paths):
        with open(file_path, 'r') as f:
            geojson = json.load(f)
            for feature in geojson.get("features", []):
                geometry = feature.get("geometry", {})
                properties = feature.get("properties", {})
                value = properties.get("value")  # change if your key is different

                if geometry.get("type") == "Polygon":
                    coords = geometry.get("coordinates", [])
                    if coords and coords[0]:  # outer ring exists
                        raw_coord = coords[0][0]  # first coordinate of outer ring
                        # Format coordinate to 10 decimal places, pad with zeros if necessary
                        coord_key = tuple([f"{c:.10f}" for c in raw_coord])
                        coordinate_data[coord_key][idx] = value

    # Convert to DataFrame
    df = pd.DataFrame([
        {"Coordinate": coord, **{file_labels[i]: values[i] for i in range(len(file_labels))}}
        for coord, values in coordinate_data.items()
    ])

    # Optional: remove rows where all values are None
    # df = df.dropna(subset=file_labels, how='all')

    # Show result
    # Ensure the output directory exists
    output_dir = os.path.join(output_folder, f"04_inverse/{stat}")
    os.makedirs(output_dir, exist_ok=True)
    df.to_json(os.path.join(output_dir, f"{filename}_inverse.json"), orient='records', indent=4)

In [20]:

for stat in stats:
    inverseGeojson(f"TAS_{stat}_cdd", stat)
    inverseGeojson(f"TAS_{stat}_cfd", stat)
    inverseGeojson(f"TAS_{stat}_hdd", stat)
    inverseGeojson(f"TAS_{stat}_hwfreq", stat)
    inverseGeojson(f"TAS_{stat}_hwlen", stat)
    inverseGeojson(f"TAS_{stat}_maxT", stat)
    inverseGeojson(f"TAS_{stat}_minT", stat)


### Merging JSONS for graph

In [21]:
def graphData(stat):
    file_paths = {
        "TAS_cdd": os.path.join(output_folder, "04_inverse", stat,f"TAS_{stat}_cdd_inverse.json"),
        "TAS_cfd":os.path.join(output_folder, "04_inverse", stat, f"TAS_{stat}_cfd_inverse.json"),
        "TAS_hdd":os.path.join(output_folder, "04_inverse", stat, f"TAS_{stat}_hdd_inverse.json"),
        "TAS_hwfreq":os.path.join(output_folder, "04_inverse", stat, f"TAS_{stat}_hwfreq_inverse.json"),
        "TAS_hwlen":os.path.join(output_folder, "04_inverse", stat, f"TAS_{stat}_hwlen_inverse.json"),
        "TAS_maxT":os.path.join(output_folder, "04_inverse", stat, f"TAS_{stat}_maxT_inverse.json"),
        "TAS_minT":os.path.join(output_folder, "04_inverse", stat, f"TAS_{stat}_minT_inverse.json"),
    }
    
    data_by_metric = {}

    # Load each file and organize by coordinate
    for metric, path in file_paths.items():
        with open(path, 'r') as f:
            records = json.load(f)
            for entry in records:
                # Convert coordinate list to tuple for use as a dictionary key
                coord_key = tuple(entry["Coordinate"])
                if coord_key not in data_by_metric:
                    data_by_metric[coord_key] = {}
                # Extract values for 1990 to 2080 in 10-year steps
                values = [entry.get(str(year)) for year in range(1990, 2090, 10)]
                data_by_metric[coord_key][metric] = values

    # Convert coordinate tuples to strings for JSON serialization
    final_output = {str(k): v for k, v in data_by_metric.items()}

    # Save the final merged dictionary as a new JSON file
    output_file = os.path.join(output_folder, f"05_graphData/{stat}.json")
    with open(output_file, 'w') as out_file:
        json.dump(final_output, out_file, indent=2)

In [23]:
json_folder = os.path.join(output_folder, "05_graphData")

# Clear any files within json_folder before proceeding
for file in glob.glob(os.path.join(json_folder, "*")):
    os.remove(file)

for stat in stats:
    graphData(stat)
json_files = glob.glob(os.path.join(json_folder, "*.json"))
merged_json = {}
for file_path in json_files:
    key = os.path.splitext(os.path.basename(file_path))[0]
    with open(file_path, "r") as f:
        merged_json[key] = json.load(f)

with open(os.path.join(json_folder, "all_data.json"), "w") as out_file:
    json.dump(merged_json, out_file, indent=2)

### Merge the two all_data.json files

In [24]:
pr_data_path = './processed/pr/05_graphData/all_data.json'
tas_data_path = './processed/tas/05_graphData/all_data.json'
output_path = './processed/all_data.json'

# Load both JSON files
with open(pr_data_path, 'r') as f:
    pr_json = json.load(f)
with open(tas_data_path, 'r') as f:
    tas_json = json.load(f)

# Merge the two dictionaries
merged_json = {}

# Add all keys from tas_json
for stat_key, stat_data in tas_json.items():
    merged_json[stat_key] = stat_data

# Add all keys from pr_json (if key exists, merge metrics; else, add new)
for stat_key, stat_data in pr_json.items():
    if stat_key in merged_json:
        # Merge metrics for each coordinate
        for coord_key, metrics in stat_data.items():
            if coord_key in merged_json[stat_key]:
                merged_json[stat_key][coord_key].update(metrics)
            else:
                merged_json[stat_key][coord_key] = metrics
    else:
        merged_json[stat_key] = stat_data

# Save merged result
with open(output_path, 'w') as out_file:
    json.dump(merged_json, out_file, indent=2)